In [1]:
import sys, os
sys.path.append('../')
#from models.util import jsma_symbolic
import tensorflow as tf
import keras.backend as K
import numpy as np
from keras.utils import np_utils


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Create TF session and set as Keras backend session
gpu_options = tf.GPUOptions(allow_growth=False)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)
#print "Created TensorFlow session and set Keras backend."

Using TensorFlow backend.


In [ ]:
from sys import getsizeof
import gc
def load_data():
    ben_data = np.load('../data/ben_matrix.npy')
    mal_data = np.load('../data/mal_matrix.npy')
    
    ben_lab = np.zeros((ben_data.shape[0]), dtype=np.int8)
    mal_lab = np.ones((mal_data.shape[0]), dtype=np.int8)
    ben_lab = np_utils.to_categorical(ben_lab, 2)
    mal_lab = np_utils.to_categorical(mal_lab, 2)
    
    print (ben_data.shape, mal_data.shape)
    print (getsizeof(ben_data), getsizeof(mal_data), getsizeof(ben_lab), getsizeof(mal_lab))
    
    X = np.concatenate((ben_data, mal_data), axis=0)
    Y = np.concatenate((ben_lab, mal_lab), axis=0)
    del ben_data, mal_data, ben_lab, mal_lab
    gc.collect()
    
    num_samples = X.shape[0]
    
    print (num_samples)
    num_val = int(.05*num_samples)
    num_test = int(.05*num_samples)
    num_train = num_samples - num_val - num_test
    
    reshuffle_idx = np.random.choice(range(X.shape[0]), num_samples,replace=False)
    X = X[reshuffle_idx]
    Y = Y[reshuffle_idx]
    
    X_train = X[:num_train]
    Y_train = Y[:num_train]
    
    X_val = X[num_train:num_train+num_val]
    Y_val = Y[num_train:num_train+num_val]
    
    X_test = X[num_train+num_val:]
    Y_test = Y[num_train+num_val:]
    
    print (X_train.shape, X_val.shape, X_test.shape)
    print (Y_train.shape, Y_val.shape, Y_test.shape)
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [ ]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data()

((123453, 545333), (5560, 545333))
(67322994961, 3032051592, 112, 112)


In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten

def build_model():
    layers = [
        Dense(64, input_shape=(545333,)),
        Activation('relu'),
        Dropout(0.5),
        Dense(128),
        Activation('relu'),
        Dropout(0.5),
        Dense(64),
        Activation('relu'),
        Dropout(0.5),
        Dense(2),
        Activation('softmax')
    ]
    
    
    model = Sequential()
    for layer in layers:
        model.add(layer)
    return model

In [ ]:
#model = build_model()
#model.compile(loss='categorical_crossentropy',
#              optimizer='adadelta',
#              metrics=['accuracy'])
#model.fit(
#    X_train, 
#    Y_train,
##    epochs=5,
 #   batch_size=128,
#    validation_data=(X_val, Y_val),
#    verbose=1,
#    shuffle=True
#)

#model_json = model.to_json()
#with open('../trained_models/drebin.json', "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights('../trained_models/drebin.weights')
#print("Saved model to disk")

In [ ]:
model = build_model()
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
json_file = open('../trained_models/drebin.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# load weights into new model
model.load_weights('../trained_models/drebin.weights')
print("Loaded model from disk")

In [ ]:
### define model loss
def model_loss(y, prediction):
    op = prediction.op
    if "softmax" in str(op).lower():
        logits, = op.inputs
        #print "logit"
    else:
        logits = prediction
        #print "model"        
    out = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels =y)
    return out

def FGSM(X, Y, model, sess, batch_size=1):
    x = tf.placeholder(dtype= tf.float32, shape=(1,545333))
    y_ = tf.placeholder(dtype = tf.float32, shape=(1,2))
    y = model(x)
    loss = model_loss(y_, y)
    gradients = tf.gradients(loss,x)
    
    adv = tf.sign(gradients)
    #Get all non negative indices
    neg = tf.constant(-1, shape=(1, 545333),dtype=tf.float32)
    pos_idx = tf.cast(tf.not_equal(adv, neg), dtype=tf.float32)
    #Add 1 to all indices that will make the loss go up
    x_adv_raw = x + pos_idx
    #Clip feature vectors to be 0 or 1
    x_adv = tf.mod(x_adv_raw, 2)
    
    nb_batches = int(np.ceil(len(X) / batch_size))
    X_adv = np.zeros_like(X)
    for i in range(nb_batches):
        X_adv[i*batch_size:(i+1)*batch_size] = sess.run(
            x_adv,
            feed_dict={
                x: X[i*batch_size:(i+1)*batch_size],
                y_: Y[i*batch_size: (i+1)*batch_size],
                K.learning_phase():0
            }
        )
        
    return X_adv   

In [ ]:
max_iters=100
nb_classes = 2
nb_features=545333
increase = True
theta=1
gamma=.05
clip_min=0.0
clip_max=1.0

In [ ]:
def tf_print(tensor, transform=None):

    # Insert a custom python operation into the graph that does nothing but print a tensors value 
    def print_tensor(x):
        # x is typically a numpy array here so you could do anything you want with it,
        # but adding a transformation of some kind usually makes the output more digestible
        print(x if transform is None else transform(x))
        return x
    log_op = tf.py_func(print_tensor, [tensor], [tensor.dtype])[0]
    with tf.control_dependencies([log_op]):
        res = tf.identity(tensor)

    # Return the given tensor
    return res


# Same loop variables as above
def modify_input(x_in, domain_in, y_in):
    preds = model(x_in)
    preds_onehot = tf.one_hot(tf.argmax(preds, axis=1), depth=nb_classes)
    
    # create the Jacobian graph
    list_derivatives = []
    for class_ind in xrange(nb_classes):
        derivatives = tf.gradients(preds[:, class_ind], x_in)
        list_derivatives.append(derivatives[0])
    grads = tf.reshape(tf.stack(list_derivatives),
                           shape=[nb_classes, -1, nb_features])

    # Compute the Jacobian components
    # To help with the computation later, reshape the target_class
    # and other_class to [nb_classes, -1, 1].
    # The last dimention is added to allow broadcasting later.
    target_class = tf.reshape(tf.transpose(y_in, perm=[1, 0]),
                                  shape=[nb_classes, -1, 1])
    other_classes = tf.cast(tf.not_equal(target_class, 1), tf.float32)

    grads_target = tf.reduce_sum(grads * target_class, axis=0)
    grads_other = tf.reduce_sum(grads * other_classes, axis=0)

    # Remove the already-used input features from the search space
    # Subtract 2 times the maximum value from those values so that
    # they won't be picked later
    increase_coef = (4 * int(increase) - 2) \
            * tf.cast(tf.equal(domain_in, 0), tf.float32)

    target_tmp = grads_target
    target_tmp -= increase_coef \
            * tf.reduce_max(tf.abs(grads_target), axis=1, keep_dims=True)

    other_tmp = grads_other
    other_tmp += increase_coef \
            * tf.reduce_max(tf.abs(grads_other), axis=1, keep_dims=True)

    # Create a mask to only keep features that match conditions
    if increase:
        scores_mask = ((target_tmp > 0) & (other_tmp < 0))
    else:
        scores_mask = ((target_tmp < 0) & (other_tmp > 0))

   
    scores = tf.cast(scores_mask, tf.float32) * (-target_tmp * other_tmp)
    # Extract the best pixel
    best = tf.argmax(
                    tf.reshape(scores, shape=[-1, nb_features]),
                    axis=1)[0]
   
    p1_one_hot = tf.one_hot(best, depth=nb_features)
 
    # Update the search domain
    domain_out = domain_in - p1_one_hot

    # Apply the modification to the images
    if increase:
        #with tf.control_dependencies([p1_one_hot]):
        x_out = tf.minimum(clip_max, x_in + p1_one_hot)
    else:
        x_out = tf.maximum(clip_min, x_in - p1_one_hot)
    
    return x_out, domain_out, preds

In [ ]:
def jsma(X, Y, model, sess, batch_size=1):
    x = tf.placeholder(dtype=tf.float32, shape=(1,545333))
    y = tf.placeholder(dtype = tf.float32, shape=(1,2))
    search_domain = tf.placeholder(dtype=tf.float32, shape=(1,545333))
    
    x_out, domain_out, preds = modify_input(x, search_domain, y)
    
    #Iterate over entire batch
    nb_batches = int(np.ceil(len(X) / batch_size))
    X_adv = np.zeros_like(X)
    avg_iterations = 0
    for i in range(nb_batches):
        x_i = X[i*batch_size:(i+1)*batch_size]
        y_ = Y[i*batch_size: (i+1)*batch_size]
        d_i = (x_i != 1).astype(int)  
        for j in range(max_iters): 
            feed_dict={
                    x: x_i,
                    y: y_, 
                    search_domain: d_i,
                    K.learning_phase():0 
            }
            
            #Get updated values for input and search domain
            x_i, d_i, pred = sess.run([x_out, domain_out, preds], feed_dict=feed_dict)
            if (np.argmax(y_) == np.argmax(pred)):
                break
        #Update return matrix
        X_adv[i*batch_size:(i+1)*batch_size] = x_i


    return X_adv 


In [ ]:
y_tar = (Y_test == 0).astype(int)

In [ ]:
X_jsma = jsma(X_test, y_tar, model, sess)

In [ ]:
model.evaluate(X_jsma, Y_test)

In [ ]:
model.evaluate(X_test, Y_test)